# Lever-center trial unique data

We want to get single trials data at the lever.

We can use the navPath that includes all the journey with at least a lever press.

For each neuron, we want to have a matrix with rate as a function of journey (navPath) characteristics

* Distance from lever
* Angle from lever to animal when the animal is close to the lever

We can then use these matrices to compare activity during trials with accurate and inacurate homing.

In [1]:
%load_ext autoreload
%autoreload 2
import os


%run ../setup_project.py
%run ../neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy")

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023/autopi_ca1
Reading /ext_drives/d80/Jazi_etal_2023/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:10<00:00,  3.90it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [2]:
%%time
myProjectWithTrials = loadMyProjectWithTrials(myProject)

Loading: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/myProjectWithTrials.pickle
CPU times: user 4.71 s, sys: 1.72 s, total: 6.43 s
Wall time: 6.43 s


## Replicate standard map findings

We will first calculate a rate-lever_distance histogram and a polar plot of the firing around the lever. 

We can use this to validate our analysis relative to standard maps.

We start with a single sesson 

In [3]:
def rateHistoAroundLever(na,navPathType="all",light="light",nLeverMin=1,behavioralVariable="animalAngleFromLever",maxDistFromLever=15,nBins=36):
    """
    Create a rate x orientation around lever histogram
    
    The histogram is stored in a dictionary within na
    """
    names = na.getNavPathNames(navPathType=navPathType,light=light,nLeverMin=nLeverMin)
    naIfr,naInav = na.getSingleNavPathData(navPathNames=names)
    indices =naInav.targetDistance<maxDistFromLever # only keep near the lever
    res = stats.binned_statistic(naInav[behavioralVariable][indices],naIfr[indices],bins=np.arange(-np.pi,np.pi,np.pi/nBins))
    na.navPathResultsDict["histo"][navPathType+"_"+light+"_"+behavioralVariable] = {"histo":res[0],"bins":res[1]}


In [4]:
def leverCenteredIFRMaps(ses,sSes):
    #
    print(ses.name)
    
    # load raw data from file
    ifr, navs, inav = load_ifr_behavior(ses)
    
    # create a list of NeuronAutopi object
    naList=[]
    for i, n in enumerate(sSes.cg.neuron_list):
        neuronIndex=i
        cluId = sSes.name+"_"+n.spike_train.name
        na = NeuronAutopi(name = cluId, ifr = (ifr[0][neuronIndex,:],ifr[1]),navPathSummary=navs,iNavPath=inav,trialElectro=ses.trialList[0])
        naList.append(na)
   
    # add a behavioral variable from existing ones
    inav["animalAngleFromLever"]=np.arctan2(inav.targetToAnimalY,inav.targetToAnimalX)
    
    # create the matrix with the rate * lever distance * journey
    # create the histogram with rate * lever distance
    # create the histogram with rate * orientation around lever
    for light in ["light","dark"]:
        print("light:", light)
        for i, na in enumerate(naList):
            na.navPathBehavioralMatrix(navPathType="all",light=light,nLeverMin = 1,behavioralVariable = "targetDistance",bins=np.arange(0,70,2)) # per journey
            na.navPathBehavioralHistogram(navPathType="all",light=light,nLeverMin = 1,behavioralVariable = "targetDistance",bins=np.arange(0,40,2)) # all journeys together
            rateHistoAroundLever(na,navPathType="all",light=light,nLeverMin=1,behavioralVariable="animalAngleFromLever",maxDistFromLever=15,nBins=36)  

    
    sesRes={}
    for na in naList:
        sesRes[na.name] = na.navPathResultsDict
    
    return sesRes

In [5]:
ses = myProjectWithTrials.sessionList[0]
sSes = sSesList[0]

In [6]:
res = leverCenteredIFRMaps(ses,sSes)

mn5824-20112020-0107
light: light
light: dark


In [7]:
%time
res = [  leverCenteredIFRMaps(ses,sSes) for ses,sSes in zip(myProjectWithTrials.sessionList[:],sSesList[:])]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
mn5824-20112020-0107
light: light
light: dark
mn5824-22112020-0107
light: light
light: dark
mn5824-24112020-0107
light: light
light: dark
mn5824-02122020-0106
light: light
light: dark
mn711-28012021-0106
light: light
light: dark
mn711-30012021-0106
light: light
light: dark
mn711-31012021-0107
light: light
light: dark
mn711-01022021-0107
light: light
light: dark
mn711-02022021-0108
light: light
light: dark
mn711-03022021-0107
light: light
light: dark
mn711-04022021-0107
light: light
light: dark
mn2739-11022021-0107
light: light
light: dark
mn2739-15022021-0105
light: light
light: dark
mn2739-16022021-0106
light: light
light: dark
mn2739-17022021-0106
light: light
light: dark
mn2739-21022021-0106
light: light
light: dark
mn3246-09042021-0106
light: light
light: dark
mn3246-10042021-0106
light: light
light: dark
mn3246-12042021-0106
light: light
light: dark
mn3246-14042021-0106
light: light
light: dark
mn1173-02052021-0107
li

`res` is a list of dictionaries that we need to merge.

In [8]:
newRes={}
for d in res:
    newRes.update(d)
print("Number of neurons: {}".format(len(newRes)))

Number of neurons: 1080


In [9]:
fn=myProject.dataPath+"/results/leverIFRDict.pickle"
print("saving:",fn)
with open(fn, 'wb') as handle:
    pickle.dump(newRes, handle)

saving: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/leverIFRDict.pickle


We now have a dictionary with all our neurons.